# Differential Phase Shift QKD protocol Simulator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
import math

<div style="text-align: center;">
    <img src="Figures/DPS/dps_fig.png" width="800">
</div>

## Introduction to QKD

Quantum Key Distribution (QKD) allows two parties (commonly called Alice and Bob) to generate a shared, secret random key, secure against any eavesdropping attempts, thanks to the laws of quantum physics.

## DPS-QKD Protocol Overview

**Differential Phase Shift (DPS) QKD** encodes information in the phase difference between consecutive coherent light pulses.

- Alice sends a train of weak coherent pulses with random phase modulations.
- Bob uses an interferometer to measure the phase differences and extract the key.
- Security is based on the inability of an eavesdropper (Eve) to perfectly distinguish the phase differences without disturbing the quantum states.

### Key Parameters

- Mean photon number per pulse ($\mu$)
- Channel transmittance ($\eta_{ch}$)
- Detector efficiency ($\eta_{det}$)
- Dark count probability ($p_{dark}$)
- Bit error rate ($e_{bit}$)
- Fiber loss coefficient ($\alpha$)
- Distance ($d$)

---

<div style="text-align: center;">
    <img src="Figures/DPS/QKD_sim-DPSqkd-steps.png" width="800">
</div>

## DPSQKDSimulator: Design and Code Walkthrough

Below is the implementation and explanation of the `DPSQKDSimulator` class. This simulator allows you to explore how the secret key rate and error rates depend on system parameters.

#### Parameter Explanations:

- **nem:** Total number of blocks sent by Alice; determines overall statistics.
- **repetition_rate ($f_{rep}$):** Pulses sent per second.
- **mu ($\mu$):** Mean photon number per pulse; controls security and key rate.
- **ebit ($e_{bit}$):** Bit error rate; includes all error sources.
- **eta_det ($\eta_{det}$):** Detector quantum efficiency.
- **dark_count ($p_{dark}$):** Probability of a dark count (false positive) per detection window.
- **alpha ($\alpha$):** Fiber loss coefficient, usually in dB/km.
- **distance ($d$):** Channel length in km.
- **delta_bs1, delta_bs2:** Deviations from ideal beamsplitter operation.
- **t_prob:** Probability of selecting a code event for key generation.


In [ ]:
class DPSQKDSimulator:
    """
    Simulator for Differential Phase Shift (DPS) Quantum Key Distribution protocol.
    """
    def __init__(self, 
                 nem=10**6,              # Number of blocks sent by Alice
                 repetition_rate=1e9,    # Pulse repetition rate (Hz)
                 mu=0.1,                 # Mean photon number per pulse
                 ebit=0.01,              # Bit error rate
                 eta_det=0.1,            # Detector efficiency
                 dark_count=1e-7,        # Dark count probability per detection window
                 alpha=0.2,              # Fiber loss coefficient (dB/km)
                 distance=50,            # Channel distance (km)
                 delta_bs1=0.005,        # BS1 transmittance deviation from ideal 50%
                 delta_bs2=0.005,        # BS2 transmittance deviation from ideal 50%
                 t_prob=0.5):            # Probability for code event selection

#### Derived Quantities

- **Channel transmittance:**
  $$
  \eta_{ch} = 10^{-\alpha \cdot d / 10}
  $$
- **Overall transmittance:**
  $$
  \eta = \eta_{ch} \cdot \eta_{det}
  $$



### Key Functions and Mathematical Background

#### 1. Poisson Probability

For a mean $\lambda$, the probability of $k$ photons:
$$
P(k; \lambda) = \frac{e^{-\lambda} \lambda^k}{k!}
$$

In [ ]:
def _calculate_lambda(self, eta1, eta2):
        """
        Calculate the lambda factor according to equation (16) in the paper.
        """
        term1 = 1 - (1 - eta1) * eta2
        term2 = np.sqrt((term1)**2 - 4 * eta1 * (1 - eta1) * (1 - eta2)**2)
        numerator = term1 + term2
        denominator = 2 * (1 - eta1) * (1 - eta2)**2
        return numerator / denominator
    
def _calculate_poisson_prob(self, k, mean):
    """
    Calculate Poisson probability mass function for exactly k events with given mean.
    """
    return np.exp(-mean) * (mean ** k) / math.factorial(k)

def _calculate_qn(self, n, mu=None):
    """
    Calculate qn, the probability of emitting exactly n photons in the block.
    """
    if mu is None:
        mu = self.mu
        
    # Each block has 3 pulses with mean photon number mu per pulse
    total_mu = 3 * mu
    return self._calculate_poisson_prob(n, total_mu)

#### 2. Gain and Yield Calculations

- **Yield for $n$ photons:**
  - Vacuum ($n=0$): $Y_0 = p_{dark}$
  - $n > 0$: $Y_n = 1 - (1 - \eta)^n$
- **Gain:**
  $$
  Q = \sum_{n=0}^{n_{max}} P(n) \cdot Y_n
  $$

In [ ]:
def _calculate_yields_and_gains(self, mu=None, eta=None):
        """
        Calculate yields and gains for different photon number components.
        Based on GLLP security analysis framework.
        """
        if mu is None:
            mu = self.mu
        
        if eta is None:
            eta = self.eta
            
        # Calculate yields (conditional detection probabilities) for different photon numbers
        Y = {}
        for n in range(6):  # Consider up to 5 photons (sufficient for typical μ values)
            if n == 0:
                # Vacuum yield is just the dark count rate
                Y[n] = self.dark_count
            else:
                # n-photon yield is the probability of at least one photon being detected
                Y[n] = 1 - (1 - eta)**n
        
        # Calculate gain for each photon number component
        gain = {}
        total_gain = 0
        block_mu = 3 * mu  # For a 3-pulse block
        
        for n in range(6):
            # Probability of emitting n photons * yield of n-photon states
            p_n = self._calculate_poisson_prob(n, block_mu)
            gain[n] = p_n * Y[n]
            total_gain += gain[n]
        
        return Y, gain, total_gain

#### 3. Phase Error Rate (GLLP Framework)

- **Vacuum:** $e_{vacuum} = 0.5$
- **Single-photon:** $e_{single} = e_{bit}$
- **Multi-photon:** $e_{multi} = 0.5$ (worst case)

Weighted sum:
$$
e_{phase} = \frac{Q_0 e_{vacuum} + Q_1 e_{single} + \sum_{n \geq 2} Q_n e_{multi}}{Q_{total}}
$$

#### 4. Secret Key Rate ($R$)

- **Shannon entropy:**
  $$
  h(p) = -p\log_2 p - (1-p)\log_2(1-p)
  $$
- **Key length per block:**
  $$
  \ell = N_{code} - N_{EC} - N_{PA}
  $$
  - $N_{code}$: Number of code events
  - $N_{EC}$: Cost of error correction
  - $N_{PA}$: Privacy amplification cost

- **Secret key rate:**
  $$
  R = \frac{\ell}{N_{pulses}}
  $$
- **Final SKR (bits/s):**
  $$
  R_{bps} = R \cdot f_{rep}
  $$

In [ ]:
def _calculate_phase_error_rate(self, gain, mu=None):
        """
        Calculate upper bound on phase error rate using proper security proof.
        """
        if mu is None:
            mu = self.mu
            
        # QBER for single-photon component
        e1 = self.ebit
        
        # Phase error rate contribution from different sources
        e_vacuum = 0.5  # Vacuum contribution (random)
        e_single = e1   # Single-photon contribution
        e_multi = 0.5   # Multi-photon contribution (worst case)
        
        # Overall phase error rate with proper weighting
        total_gain = sum(gain.values())
        if total_gain > 0:
            e_phase = (gain[0] * e_vacuum + gain[1] * e_single + 
                      sum([gain[n] * e_multi for n in range(2, 6)])) / total_gain
        else:
            e_phase = 0.5
            
        return e_phase
    
    def calculate_secret_key_rate(self, mu=None):
        """
        Unified secret key rate calculation that works for both mu and distance analysis.
        Based on rigorous GLLP security analysis for weak coherent pulse QKD.
        """
        if mu is None:
            mu = self.mu
            eta = self.eta
        else:
            # If mu is explicitly provided, we keep the current eta (distance)
            eta = self.eta
        
        # Calculate yields and gains
        Y, gain, Q_gllp = self._calculate_yields_and_gains(mu, eta)
        
        # Phase error rate based on GLLP analysis
        e_phase = self._calculate_phase_error_rate(gain, mu)
        
        # For compatibility with the original distance behavior,
        # we also calculate Q using the original formula
        Q_original = 2 * eta * mu * np.exp(-2 * eta * mu)
        
        # Blend the two models with a weighting that depends on whether we're
        # analyzing mu or distance dependency
        # For mu analysis (bell curve), we want GLLP to dominate
        # For distance analysis, we want original formula to dominate
        # We can determine this by checking if mu was explicitly provided
        
        # Calculate correction factor based on distance to favor original formula
        # as distance increases
        distance_factor = min(1.0, self.distance / 100)
        
        if mu == self.mu:  # distance analysis (no explicit mu provided)
            # Use a blend that favors the original formula
            Q = Q_original
            
            # Calculate upper bound on phase error rate according to original equation (15)
            q1 = self._calculate_qn(1, mu)
            q2 = self._calculate_qn(2, mu)
            q3 = self._calculate_qn(3, mu)
            e_U_ph = (self.lambda_factor * (self.ebit + np.sqrt(q1 * q3) / Q + 2 * self.delta_bs2) 
                     + q2 / Q)
            
            # Number of code events
            Ncode = self.t_prob * self.nem * Q
            
            # Cost of error correction - based on Shannon limit
            NEC = Ncode * self._shannon_entropy(self.ebit)
            
            # Amount of privacy amplification
            NPA = Ncode * self._shannon_entropy(e_U_ph)
            
            # Secret key length
            ell = Ncode - NPA - NEC
            
            # Secret key rate per emitted pulse
            R = ell / (3 * self.nem)
            
        else:  # mu analysis (explicit mu provided)
            # For mu analysis, use GLLP framework
            Q = Q_gllp
            
            # Calculate single-photon and multi-photon contributions
            p1 = self._calculate_qn(1, mu)
            Y1 = Y[1]
            
            # Calculate secret fraction based on GLLP security analysis
            r = max(0, gain[1]/Q * (1 - self._shannon_entropy(e_phase)) - 
                    self._shannon_entropy(self.ebit))
            
            # Final key rate (bits per pulse)
            R = r * Q
        
        # Convert to bits per second
        R_bps = R * self.repetition_rate
        
        return max(0, R_bps)

#### 5. QBER Calculation

QBER includes both signal and noise contributions:
$$
QBER = \frac{0.5 \cdot p_{dark} + e_{bit} \cdot P_{signal} + 0.25 \cdot P_{multi} \cdot \eta}{P_{signal} + p_{dark}}
$$

In [ ]:
def _shannon_entropy(self, p):
        """
        Shannon entropy function h(p) = -p*log2(p) - (1-p)*log2(1-p)
        """
        if p <= 0 or p >= 1:
            return 0
        return -p * np.log2(p) - (1-p) * np.log2(1-p)
    

def calculate_qber(self):
    """
    Calculate the expected QBER based on dark counts and signal.
    """
    # Signal detection probability
    signal_prob = self.eta * self.mu * np.exp(-self.mu)  # Changed formula
    
    # Multi-photon contribution that increases with higher mu
    multi_photon_error = 1 - np.exp(-self.mu) - self.mu * np.exp(-self.mu)
    
    # Noise due to dark counts
    noise_prob = self.dark_count
    
    # Base error rate from imperfect interference
    base_error = self.ebit
    
    # QBER calculation with multi-photon contribution that increases with μ
    if signal_prob + noise_prob > 0:
        # Add a term that increases with mu to get increasing QBER
        qber = (0.5 * noise_prob + base_error * signal_prob + 
                0.25 * multi_photon_error * self.eta) / (signal_prob + noise_prob)
    else:
        qber = 0.5  # No signal, only random noise
        
    return qber

# QBER Calculation

<div style="text-align: center;">
    <img src="Figures/DPS/QKD_sim-Copy of DPSqkd-qber.png" width="700">
</div>

# SKR Calculation

<div style="text-align: center;">
    <img src="Figures/DPS/QKD_sim-DPSqkd-skr.png" width="1000">
</div>

<a name="plotting"></a>
# **Plotting and Results Visualization**

The module provides functions to visualize QBER and SKR as a function of mean photon number $\mu$ and channel distance $d$.

In [ ]:
from ThOPqkdsim.sim10_5 import *

def run_cow_qkd_simulation():
    sim = DPSQKDSimulator(
        nem=10**6,               # Number of blocks sent
        repetition_rate=1e9,     # 1 GHz repetition rate
        mu=0.1,                  # Mean photon number
        ebit=0.01,               # Bit error rate
        eta_det=0.1,             # Detector efficiency
        dark_count=1e-7,         # Dark count probability
        alpha=0.2,               # Fiber loss (dB/km)
        distance=50,             # Distance in km
        delta_bs1=0.005,         # BS1 deviation
        delta_bs2=0.005,         # BS2 deviation
        t_prob=0.5               # Probability for code event
    )

    # Print simulation summary
    sim.plot_qber_vs_mu()
    sim.plot_qber_vs_distance()
    sim.plot_skr_vs_mu(mu_range=(0.001, 0.5))  
    sim.plot_skr_vs_distance()


    plt.show()

In [ ]:
run_cow_qkd_simulation()

## Conclusion: Differential Phase Shift (DPS) QKD Simulator

The **Differential Phase Shift (DPS) QKD Simulator** presented in this module provides an accessible yet rigorous framework for exploring the performance and security of DPS-based quantum key distribution systems. By encapsulating both the fundamental physics and the practical engineering parameters—such as mean photon number, detector efficiency, channel loss, and dark counts—the simulator enables users to:

- **Visualize and analyze** how the secret key rate (SKR) and quantum bit error rate (QBER) depend on both the mean photon number and the transmission distance.
- **Apply security proofs** (GLLP and original DPS-specific formulas) to obtain realistic bounds on key rates.
- **Optimize protocol parameters** (e.g., mean photon number) for maximal secure throughput.
- **Demonstrate trade-offs** inherent in real-world QKD deployments, such as the impact of increasing channel loss or detector imperfections.

Through interactive plotting and parameter adjustments, users can gain intuition about the operational regime of DPS-QKD, making this simulator a useful tool for both education and protocol research.

---

## Future Work

While the current simulator offers a comprehensive platform for DPS-QKD analysis, several avenues remain open for enhancement and further research:

1. **Integration of Advanced Attack Models:**  
   Extend the simulator to include explicit modeling of eavesdropping strategies (e.g., photon number splitting attacks, optimal quantum cloning), and simulate countermeasures such as decoy states.

2. **Finite-Key Effects:**  
   Incorporate finite-key analysis, accounting for statistical fluctuations in realistic, finite-length key exchanges, which are critical for practical deployments.

3. **Experimental Imperfections:**  
   Model additional sources of system imperfections, such as polarization drift, time jitter, and afterpulsing in detectors, for even more realistic performance predictions.

4. **Multi-User and Networked Scenarios:**  
   Expand the simulation to address multi-user QKD networks, key relay, and trusted repeater architectures.

5. **Comparison with Other Protocols:**  
   Build out unified interfaces and visualizations to directly compare DPS-QKD performance with BB84, COW, and other QKD protocols under identical conditions.

6. **User Interface Improvements:**  
   Develop a graphical or web-based front end to make the simulator accessible to users without programming experience and facilitate classroom demonstrations.

7. **Integration with Experimental Data:**  
   Enable importing experimental or field trial data to benchmark and validate simulation results.
